# This notebook is in use for captsone project for IBM Data Science Proffessional Certifcate.

In [1]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


# WEEK 3 ASSIGNMENT

# PART 1: Webscraping and Preparing Dataframe

Using read_html function I webscrape from the given url. From the webscape there is 3 dataframes so I use the first one by coding [0]. 

In [46]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

df=pd.read_html(url, header=0, flavor='html5lib')[0]

Gettting rid of not assigned label in borough column.

In [51]:
df = df[df["Borough"] != "Not assigned"]
#Gettting rid of not assigned label in borough column 

Check if there's any "Not assigned" in the Neighbourhood column, combine them into one row if there is.
In this case there is none.

In [52]:
df[df["Neighbourhood"] == "Not assigned"] 
# No Neighbourhood has not assigned label

,Postal Code,Borough,Neighbourhood


Reset index.

In [56]:
df = df.reset_index(drop=True)

Get shape of the dataframe.

In [57]:
df.shape

(103, 3)

# PART 2: Getting Longitude and Latitude

Using the csv of the latitude and longitude given in Q as package was taking too long.

In [62]:
geodf=pd.read_csv("http://cocl.us/Geospatial_data")
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge our dateframes on Postal Codes.

In [65]:
dfm = pd.merge(df, geodf, on='Postal Code')
dfm.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# PART 3: Explore and Cluster Neighbourhoods

In [70]:
import requests
from sklearn.cluster import KMeans
# !pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

You are using pip version 9.0.3, however version 20.3.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


My Foursquare credentials.

In [79]:
CLIENT_ID = 'YKZ11CHJTQBZY1HOZWMQ1GGAB0RYIHOJDCMJNDDI0TMFMQMF' # your Foursquare ID
CLIENT_SECRET = 'UI4T34H2ZFIGZ45QRJYW4VXU5WYVR1BRK0DLI0KR4BJK3S1W' # your Foursquare Secret
ACCESS_TOKEN = "ZQ31XIZ01GURXSJHFUUTL1DFYYECXMOSRSRTL2HU3CINUNQS" # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 50
radius = 100

Filtering Boroughs that contain "Toronto".

In [100]:
df = dfm[dfm['Borough'].str.contains('Toronto')]
df.reset_index(drop=True, inplace=True)
df.shape

(39, 5)

Function to get category type.

In [101]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Function to get venue nearby to the given input place.

In [102]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(name, lat, lng,
                             v['venue']['name'],
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 
                             'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [103]:
venues = getNearbyVenues(df['Neighbourhood'], df['Latitude'], df['Longitude'])
                                  

In [104]:
venues.shape

(1175, 7)

Converting venue categories into dummy integer variables to allow easier analysis.

In [115]:
dff = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
dff.rename(columns={'Neighbourhood':'Neighbourhood (category)'}, inplace=True)
dff['Neighbourhood'] = venues['Neighbourhood'] 
fixed_columns = [dff.columns[-1]] + list(dff.columns[:-1])
dff = dff[fixed_columns]
dff.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Finding frequecy percentage of category of each neigbourhood.

In [117]:
dfg = dff.groupby('Neighbourhood').mean().reset_index()
dfg.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.041667
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.058824
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0625,0.0625,0.0625,0.125,0.125,0.125,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.02,0.020000


Finding 5 most common venues.

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

topven = 5
subs = ['st', 'nd', 'rd']
columns = ['Neighbourhood']
for ind in np.arange(topven):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, subs[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
dfs = pd.DataFrame(columns=columns)
dfs['Neighbourhood'] = dfg['Neighbourhood']
for ind in np.arange(dfg.shape[0]):
    dfs.iloc[ind, 1:] = return_most_common_venues(dfg.iloc[ind, :], topven)
dfs.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Airport,Coffee Shop
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Bubble Tea Shop,Burger Joint


Performing clustering with 3 clusters.

In [136]:
k = 3
dfc = dfg.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=k, random_state=1234).fit(dfc)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Adding cluster label to our dataframe so we know which cluster each row belongs to.

In [137]:
dfs.insert(0, 'Cluster Labels', kmeans.labels_)
dfm = df
dfm = dfm.join(dfs.set_index('Neighbourhood'), on='Neighbourhood')
dfm.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Yoga Studio,Creperie,Sandwich Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Cosmetics Shop,Coffee Shop,Clothing Store,Café,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Gastropub,Coffee Shop,Seafood Restaurant,Cosmetics Shop
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Trail,Neighborhood,Health Food Store,Pub,Yoga Studio


Producing map of clustered neighbourhoods.

In [138]:
toronto_coordinates =[43.6532, -79.3832]
map_clusters = folium.Map(location=toronto_coordinates, zoom_start=11)
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
carray = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in carray]
markers_colors = []
for lat, lon, poi, cluster in zip(dfm['Latitude'], dfm['Longitude'],
                                  dfm['Neighbourhood'], dfm['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True,
                        fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters